In [32]:
#import packages
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
import pandas as pd
import spotipy.util as util
import warnings
warnings.filterwarnings("ignore")

client_id = '5e939d785eca48aa9f695ca4c390b064'
client_secret = '951453be3e4946a58f4a6eb0a658929e'
username = '316oaofnwtpaj4mwvff6w6bmacxi'

# create a dataframe to store infomation of my playlists 
my_playlist =  pd.DataFrame(columns=["id", "spotify_id", "list_name"])

# getting playlist info from spotify
client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

playlists = sp.user_playlists(username) # input your spotify account id here

# converting spotify data into dataframe
while playlists:
    for i, playlist in enumerate(playlists['items']): 
        spotifyid = playlist['id'] 
        listname = playlist['name'] 
        my_playlist = my_playlist.append({'id':i+1 , 
                        'spotify_id': spotifyid,
                        'list_name': listname
                        }, ignore_index=True)
        
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

# dataframe for song
my_song =  pd.DataFrame(columns=["list_id", "song_id","song_name","artist","popularity",'release_date']
                       )

# getting song info from each playlist
for listid in my_playlist["spotify_id"]:
    songs = []
    content = sp.user_playlist_tracks(username, listid)
    songs = content['items']
    while content['next']:
        content = sp.next(content)
        songs.extend(content['items'])
    for song in songs:
        my_song = my_song.append({"list_id" : listid,
                                  "song_id":song['track']['id'],
                                  "song_name":song['track']['name'],
                                  "artist":song['track']['artists'][0]['name'],
                                  "popularity": song['track']['popularity'],
                                  "release_date": song['track']['album']['release_date']},ignore_index=True)

# song feature dataframe
my_feature = pd.DataFrame(columns=["song_id","energy", "liveness","tempo","speechiness",
                                "acousticness","instrumentalness","danceability",
                                "duration_ms","loudness","valence",
                                "mode","key"])
# playlist songs' features
for song in my_song['song_id']:
    features = sp.audio_features(tracks = [song])[0]
    my_feature = my_feature.append({"song_id":song,
                                    "energy":features['energy'], 
                                    "liveness":features['liveness'],
                                    "tempo":features['tempo'],
                                    "speechiness":features['speechiness'],
                                    "acousticness":features['acousticness'],
                                    "instrumentalness":features['instrumentalness'],
                                    "danceability":features['danceability'],
                                    "duration_ms":features['duration_ms'],
                                    "loudness":features['loudness'],
                                    "valence":features['valence'],
                                    "mode":features['mode'],
                                    "key":features["key"],
                                 },ignore_index=True)

# merging the song info and song features dataframe
song_feature = pd.merge(my_song,my_feature,how='left',left_on='song_id', right_on='song_id')
list_song_feature = pd.merge(my_playlist,song_feature,how='left',left_on='spotify_id', right_on='list_id')

In [33]:
list_song_feature

,id,spotify_id,list_name,list_id,song_id,song_name,artist,popularity,release_date,energy,...,tempo,speechiness,acousticness,instrumentalness,danceability,duration_ms,loudness,valence,mode,key
0,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,6hDM2uj57moJd3RLYTm5rI,"Leekan (From ""Ashke"" Soundtrack)",Amrinder Gill,47,2018-07-27,0.666,...,124.159,0.0506,0.788,0,0.534,215605,-4.644,0.564,1,0
1,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,1xNN6c8eW8Mm9uz9nVqln9,Isharon Isharon Men Dil Lenewale,Asha Bhosle,48,1964-01-01,0.447,...,137.156,0.0413,0.964,0.00213,0.439,290400,-7.341,0.56,1,5
2,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,0xDdcB0gypop2DwJU2O1xG,Uden Jab Jab Zulfen Teri,Mohammed Rafi,44,1957-01-01,0.715,...,135.0,0.097,0.479,0.000633,0.525,295926,-7.658,0.961,1,6
3,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,5djTQgtCSdr7N2wOExvwc5,Wah Kya Nazare (feat. Gaby Fuego),Harnoor,44,2021-11-17,0.468,...,125.829,0.24,0.127,0,0.677,176019,-7.866,0.518,1,9
4,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,1e7gkz5Yn8d9Wkxe58qdV5,Badra Bahaar,Amit Trivedi,40,2014-02-04,0.724,...,100.051,0.0296,0.179,0.00123,0.772,214132,-6.986,0.713,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,6eEAevjmfFiu5SSvH0DRY8,Aahun Aahun,Master Saleem,45,2009-07-31,0.903,...,129.967,0.307,0.155,0.00005,0.536,290493,-5.501,0.657,0,1
713,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,6VBhH7CyP56BXjp8VsDFPZ,"Kesariya (From ""Brahmastra"")",Pritam,87,2022-07-17,0.574,...,94.009,0.0278,0.572,0,0.577,268165,-6.525,0.436,1,0
714,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,6XZWokIxs4leaSmXmU8b26,"Hum Nashe Mein Toh Nahin (From ""Bhool Bhulaiya...",Pritam,74,2022-05-09,0.736,...,85.035,0.0658,0.643,0,0.572,238252,-7.768,0.778,1,11
715,1,3TiDHuwWaWeMCRmatZRC8H,My playlist,3TiDHuwWaWeMCRmatZRC8H,0iJPMG79hgWnDHjufnoplb,Porkanda Singam EDM Version (Additional Song),Anirudh Ravichander,63,2022-07-07,0.769,...,158.953,0.0464,0.105,0.256,0.687,149434,-5.604,0.413,0,11


In [34]:
# exporting to csv file
list_song_feature.to_csv('playlist.csv')